In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append("/n/home12/binxuwang/Github/mini_edm")
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning")
import os
from os.path import join
import pickle as pkl
import einops
import torch
import torch as th
from easydict import EasyDict as edict
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
from train_edm import create_model, edm_sampler, EDM
from edm_utils import edm_sampler_inpaint, create_edm, get_default_config, create_edm_new
# from rule_utils import get_rule_img, get_obj_list, get_rule_list
# from rule_utils import check_consistent
from dataset_utils import train_data2attr_tsr,load_raw_data,load_PGM_abstract
from rule_new_utils import infer_rule_from_sample_batch, check_r3_r2_batch

In [4]:
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.dpi'] = 72
plt.rcParams['figure.figsize'] = [6.0, 4.0]
plt.rcParams['figure.edgecolor'] = (1, 1, 1, 0)
plt.rcParams['figure.facecolor'] = (1, 1, 1, 0)
# vector graphics type
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [8]:
device = "cuda"

In [14]:
# train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs_new.pt')
attr_all = np.load('/n/home12/binxuwang/Github/DiffusionReasoning/attr_all.npy')
attr_all = torch.from_numpy(attr_all)

In [15]:
# abstract RAVEN dataset
dataset_Xmean = th.tensor([1.5, 2.5, 2.5]).view(1, 3, 1, 1).to("cuda")
dataset_Xstd = th.tensor([2.5, 3.5, 3.5]).view(1, 3, 1, 1).to("cuda")

In [16]:
@torch.no_grad()
def edm_sampler_inpaint(
    edm, latents, target_img, mask, class_labels=None,
    num_steps=18, sigma_min=0.002, sigma_max=80, rho=7,
    use_ema=True, fixed_noise=False
):
    # Adjust noise levels based on what's supported by the network.
    sigma_min = max(sigma_min, edm.sigma_min)
    sigma_max = min(sigma_max, edm.sigma_max)

    # Time step discretization.
    step_indices = torch.arange(num_steps, dtype=torch.float64, device=latents.device)
    t_steps = (sigma_max ** (1 / rho) + step_indices / (num_steps - 1) * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    t_steps = torch.cat([edm.round_sigma(t_steps), torch.zeros_like(t_steps[:1])]) # t_N = 0
    initial_noise = torch.randn_like(latents)
    # Main sampling loop.
    x_next = latents.to(torch.float64) * t_steps[0]
    for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])): # 0, ..., N-1
        # x_hat = x_next
        t_hat = t_cur
        noise_perturb = initial_noise if fixed_noise else torch.randn_like(target_img)
        x_hat = (1 - mask[None, None]) * (target_img + noise_perturb * t_cur) + \
                     mask[None, None]  * x_next
        # Euler step.
        denoised = edm(x_hat, t_hat, class_labels, use_ema=use_ema).to(torch.float64)
        d_cur = (x_hat - denoised) / t_hat
        x_next = x_hat + (t_next - t_hat) * d_cur

        # Apply 2nd order correction.
        if i < num_steps - 1:
            denoised = edm(x_next, t_next, class_labels, use_ema=use_ema).to(torch.float64)
            d_prime = (x_next - denoised) / t_next
            x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)

    return x_next

In [18]:
!ls {exproot}/Wide* -d

/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnr_RAVEN10_abstract_20240211-1747
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnr_RAVEN10_abstract_onehot_20240211-1743
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnrX3_new_noattn_RAVEN10_abstract_20240412-1254
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnrX3_new_RAVEN10_abstract_20240313-1737
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnrX3_new_RAVEN10_abstract_20240315-1327
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnrX3_new_RAVEN10_abstract_20240412-0143
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnrX3_new_RAVEN10_abstract_20240412-1347
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/e

In [19]:
# expname = "WideBlnrX3_new_RAVEN10_abstract_20240315-1327"
expname = "WideBlnrX3_new_RAVEN10_abstract_20240412-1347"
exproot = r"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps"
DATASET = "RAVEN10_abstract"
expdir = join(exproot, expname)
ckptdir = join(expdir, "checkpoints")
savedir = join(expdir, "inpaint_pilot")
os.makedirs(savedir, exist_ok=True)

config_ft = get_default_config(DATASET, layers_per_block=2, 
                               model_channels=128, 
                               channel_mult=[1, 2, 4], 
                               attn_resolutions=[9, 3], 
                               spatial_matching="bilinear")
epoch = 999999 #700000
ckpt_path = join(ckptdir, f"ema_{epoch}.pth")
edm, model_EDM = create_edm_new(ckpt_path, config_ft, device) 

In [20]:
batch_size = 100

total_steps = 35
# model_kwargs = dict(y=th.zeros(batch_size, dtype=th.int, device="cuda"))
# diffusion_eval = create_diffusion(timestep_respacing="ddim200")  # default: ddim100
nreps = 5
sample_col = []
meta_col = []
rule_all_col = []
stats_col = []
# iclass = 0
for iclass in range(40):
    idx_seq = np.random.choice(12000, 3 * nreps, replace=False)
    for ireps in range(nreps):
        idxs = idx_seq[ireps*3:(ireps+1)*3]
        # idxs = np.random.randint(0, 12000, 3)#np.random.choice(12000, 3)
        attr_tsr = train_data2attr_tsr(attr_all[iclass, idxs])
        attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
        r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
        print(r3_list, "row idxs", list(idxs), rule_col)
        # assert np.all(orig_rules != -1)
        assert len(r3_list[0]) == 1
        orig_rule_id = r3_list[0][0]
        target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
        for x_pos, y_pos in [(0, 2), (1, 2), (2, 2),]:
            mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
            mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
            # samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, fixed_noise=False,
            #                                             model_kwargs=model_kwargs, device="cuda", progress=False, eta=0.0)
            latents = torch.randn(batch_size, 3, 9, 9).to(device)
            samples_inpaint = edm_sampler_inpaint(edm, latents, 
                                target_img=target_img, mask=mask_tsr,
                                num_steps=total_steps, use_ema=False,
                                fixed_noise=False).detach()
            samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
            r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
            consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
            inpaint_rule_col = np.concatenate(tuple(np.array(rule_col_ipt, dtype=object)[:, x_pos]))
            if len(inpaint_rule_col):
                uniq, counts = np.unique(inpaint_rule_col, return_counts=True)
                # sort by counts
                idx_sort = np.argsort(counts)[::-1]
                uniq_sort = uniq[idx_sort]
                counts_sort = counts[idx_sort]
            else:
                uniq_sort = []
                counts_sort = []
            print(f"Rule [{orig_rule_id}] id[{idxs[x_pos]}] Consistent rules ratio inpainting samples ({x_pos}, {y_pos}): {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")
            meta_col.append({"iclass": iclass, "idxs": list(idxs), "rule": rule_col, "x_pos": x_pos, "y_pos": y_pos})
            sample_col.append(samples_inpaint)
            rule_all_col.append((r3_list_ipt, r2_list_ipt, rule_col_ipt))
            stats_col.append({"iclass": iclass, "idxs": list(idxs), "x_pos": x_pos, "y_pos": y_pos, "consistent_r3": consistent_r3, "batch_size": batch_size, 
                              "rule_uniq": uniq_sort, "rule_cnts": counts_sort}) # "rule": rule_col, 
        print(f"iclass {iclass} done {ireps}/{nreps}")
        df = pd.DataFrame(stats_col)
        df.to_csv(join(savedir, "inpaint_rule_stats.csv"))
                
    pkl.dump(sample_col, open(join(savedir, "inpaint_sample_col.pkl"), "wb"))
    pkl.dump(meta_col, open(join(savedir, "inpaint_meta_col.pkl"), "wb"))
    pkl.dump(rule_all_col, open(join(savedir, "inpaint_rule_all_col.pkl"), "wb"))
    pkl.dump(stats_col, open(join(savedir, "inpaint_stats_col.pkl"), "wb"))
    
df = pd.DataFrame(stats_col)
df.to_csv(join(savedir, "inpaint_rule_stats.csv"))

[[0]] row idxs [11403, 2243, 8267] [[array([0]), array([0]), array([0])]]
Rule [0] id[11403] Consistent rules ratio inpainting samples (0, 2): 34/100 (0.340)
Rule [0] id[2243] Consistent rules ratio inpainting samples (1, 2): 28/100 (0.280)
Rule [0] id[8267] Consistent rules ratio inpainting samples (2, 2): 22/100 (0.220)
iclass 0 done 0/5
[[0]] row idxs [1036, 10937, 11146] [[array([0]), array([0]), array([0])]]
Rule [0] id[1036] Consistent rules ratio inpainting samples (0, 2): 52/100 (0.520)
Rule [0] id[10937] Consistent rules ratio inpainting samples (1, 2): 53/100 (0.530)
Rule [0] id[11146] Consistent rules ratio inpainting samples (2, 2): 49/100 (0.490)
iclass 0 done 1/5
[[0]] row idxs [79, 11746, 9608] [[array([0]), array([0]), array([0])]]
Rule [0] id[79] Consistent rules ratio inpainting samples (0, 2): 55/100 (0.550)
Rule [0] id[11746] Consistent rules ratio inpainting samples (1, 2): 45/100 (0.450)
Rule [0] id[9608] Consistent rules ratio inpainting samples (2, 2): 77/100 (0

In [53]:
batch_size = 100
suffix = "_unif_baseline"
total_steps = 35
# model_kwargs = dict(y=th.zeros(batch_size, dtype=th.int, device="cuda"))
# diffusion_eval = create_diffusion(timestep_respacing="ddim200")  # default: ddim100
nreps = 50
sample_col = []
meta_col = []
rule_all_col = []
stats_col = []
# iclass = 0
for iclass in range(40):
    idx_seq = np.random.choice(12000, 3 * nreps, replace=False)
    for ireps in range(nreps):
        idxs = idx_seq[ireps*3:(ireps+1)*3]
        # idxs = np.random.randint(0, 12000, 3)#np.random.choice(12000, 3)
        attr_tsr = train_data2attr_tsr(attr_all[iclass, idxs])
        attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
        r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
        print(r3_list, "row idxs", list(idxs), rule_col)
        # assert np.all(orig_rules != -1)
        assert len(r3_list[0]) == 1
        orig_rule_id = r3_list[0][0]
        target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
        for x_pos, y_pos in [(0, 2), (1, 2), (2, 2),]:
            mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
            mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
            # samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, fixed_noise=False,
            #                                             model_kwargs=model_kwargs, device="cuda", progress=False, eta=0.0)
            inpaint_part = th.cat([th.randint(-1, 7, (batch_size, 1, 3, 3), device=device),
                                   th.randint(-1, 10, (batch_size, 2, 3, 3), device=device)], dim=1)
            vacant_mask = (inpaint_part == -1).any(dim=1, keepdim=True)
            # for any entry that is -1, set the entire row (dim=1) to -1
            inpaint_part[vacant_mask.expand(-1, 3, -1, -1)] = -1
            
            samples_inpaint = attr_mtg.repeat(batch_size, 1, 1, 1)
            samples_inpaint[:, :, 3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = inpaint_part
            # raise ValueError("Not implemented")
            # latents = torch.randn(batch_size, 3, 9, 9).to(device)
            # samples_inpaint = edm_sampler_inpaint(edm, latents, 
            #                     target_img=target_img, mask=mask_tsr,
            #                     num_steps=total_steps, use_ema=False,
            #                     fixed_noise=False).detach()
            # samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
            r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
            consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
            inpaint_rule_col = np.concatenate(tuple(np.array(rule_col_ipt, dtype=object)[:, x_pos]))
            if len(inpaint_rule_col):
                uniq, counts = np.unique(inpaint_rule_col, return_counts=True)
                # sort by counts
                idx_sort = np.argsort(counts)[::-1]
                uniq_sort = uniq[idx_sort]
                counts_sort = counts[idx_sort]
            else:
                uniq_sort = []
                counts_sort = []
            print(f"Rule [{orig_rule_id}] id[{idxs[x_pos]}] Consistent rules ratio inpainting samples ({x_pos}, {y_pos}): {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")
            meta_col.append({"iclass": iclass, "idxs": list(idxs), "rule": rule_col, "x_pos": x_pos, "y_pos": y_pos})
            sample_col.append(samples_inpaint)
            rule_all_col.append((r3_list_ipt, r2_list_ipt, rule_col_ipt))
            stats_col.append({"iclass": iclass, "idxs": list(idxs), "x_pos": x_pos, "y_pos": y_pos, "consistent_r3": consistent_r3, "batch_size": batch_size, 
                              "rule_uniq": uniq_sort, "rule_cnts": counts_sort}) # "rule": rule_col, 
        print(f"iclass {iclass} done {ireps}/{nreps}")
        df = pd.DataFrame(stats_col)
        df.to_csv(join(savedir, f"inpaint_rule_stats{suffix}.csv"))
                
    pkl.dump(sample_col, open(join(savedir, f"inpaint_sample_col{suffix}.pkl"), "wb"))
    pkl.dump(meta_col, open(join(savedir, f"inpaint_meta_col{suffix}.pkl"), "wb"))
    pkl.dump(rule_all_col, open(join(savedir, f"inpaint_rule_all_col{suffix}.pkl"), "wb"))
    pkl.dump(stats_col, open(join(savedir, f"inpaint_stats_col{suffix}.pkl"), "wb"))
    
df = pd.DataFrame(stats_col)
df.to_csv(join(savedir, f"inpaint_rule_stats{suffix}.csv"))

[[0]] row idxs [3303, 1569, 11975] [[array([0]), array([0]), array([0])]]
Rule [0] id[3303] Consistent rules ratio inpainting samples (0, 2): 0/100 (0.000)
Rule [0] id[1569] Consistent rules ratio inpainting samples (1, 2): 0/100 (0.000)
Rule [0] id[11975] Consistent rules ratio inpainting samples (2, 2): 0/100 (0.000)
iclass 0 done 0/50
[[0]] row idxs [10595, 1173, 11152] [[array([0]), array([0]), array([0])]]
Rule [0] id[10595] Consistent rules ratio inpainting samples (0, 2): 0/100 (0.000)
Rule [0] id[1173] Consistent rules ratio inpainting samples (1, 2): 0/100 (0.000)
Rule [0] id[11152] Consistent rules ratio inpainting samples (2, 2): 0/100 (0.000)
iclass 0 done 1/50
[[0]] row idxs [4199, 6930, 9505] [[array([0]), array([0]), array([0])]]
Rule [0] id[4199] Consistent rules ratio inpainting samples (0, 2): 0/100 (0.000)
Rule [0] id[6930] Consistent rules ratio inpainting samples (1, 2): 0/100 (0.000)
Rule [0] id[9505] Consistent rules ratio inpainting samples (2, 2): 0/100 (0.000)

In [49]:
samples_inpaint[:2]


tensor([[[[-1, -1, -1,  4,  4,  4, -1,  4,  1],
          [-1, -1, -1, -1,  4,  4, -1,  3,  3],
          [ 4, -1, -1,  4, -1,  4,  0,  1,  5],
          [ 2,  2,  2, -1,  2,  2, -1, -1, -1],
          [ 2,  2,  2,  2,  2,  2, -1,  2,  2],
          [ 2,  2,  2,  2,  2,  2,  2, -1,  2],
          [-1,  4, -1, -1,  4,  4,  4,  4,  4],
          [ 4,  4,  4, -1,  4,  4, -1,  4,  4],
          [ 4,  4,  4, -1,  4, -1,  4,  4,  4]],

         [[-1, -1, -1,  6,  6,  6, -1,  0,  2],
          [-1, -1, -1, -1,  6,  6, -1,  5,  5],
          [ 7, -1, -1,  6, -1,  6,  6,  8,  0],
          [ 0,  2,  9, -1,  9,  9, -1, -1, -1],
          [ 9,  7,  1,  9,  0,  9, -1,  3,  4],
          [ 7,  2,  7,  0,  9,  0,  3, -1,  2],
          [-1,  3, -1, -1,  9,  3,  8,  7,  1],
          [ 4,  0,  2, -1,  1,  4, -1,  9,  0],
          [ 1,  6,  8, -1,  0, -1,  4,  6,  2]],

         [[-1, -1, -1,  8,  8,  8, -1,  2,  8],
          [-1, -1, -1, -1,  8,  8, -1,  8,  4],
          [ 9, -1, -1,  8, -1,  8,  

In [21]:
df

,iclass,idxs,x_pos,y_pos,consistent_r3,batch_size,rule_uniq,rule_cnts
0,0,"[11403, 2243, 8267]",0,2,34,100,"[0.0, 34.0, 6.0, 26.0, 25.0, 37.0, 39.0]","[34, 11, 11, 7, 6, 2, 1]"
1,0,"[11403, 2243, 8267]",1,2,28,100,"[0.0, 16.0, 35.0, 36.0, 20.0, 12.0, 26.0, 6.0,...","[28, 19, 13, 12, 11, 6, 4, 2, 2]"
2,0,"[11403, 2243, 8267]",2,2,22,100,"[33.0, 26.0, 0.0, 10.0, 22.0, 25.0, 5.0, 16.0,...","[30, 26, 22, 11, 6, 5, 5, 3, 3, 1, 1]"
3,0,"[1036, 10937, 11146]",0,2,52,100,"[0.0, 30.0, 5.0, 14.0, 15.0, 39.0, 29.0]","[52, 14, 10, 8, 6, 1, 1]"
4,0,"[1036, 10937, 11146]",1,2,53,100,"[0.0, 36.0, 22.0, 15.0, 6.0, 16.0, 10.0, 26.0,...","[53, 17, 14, 10, 7, 5, 5, 4, 3]"
...,...,...,...,...,...,...,...,...
595,39,"[1719, 3146, 11580]",1,2,11,100,"[11.0, 39.0, 30.0, 26.0, 6.0, 20.0, 16.0, 25.0...","[14, 11, 11, 11, 11, 5, 5, 3, 2, 1]"
596,39,"[1719, 3146, 11580]",2,2,12,100,"[30.0, 39.0, 15.0, 6.0, 5.0, 1.0, 25.0, 12.0, ...","[15, 12, 12, 11, 9, 9, 8, 7, 6, 5, 1]"
597,39,"[1494, 9326, 7130]",0,2,12,100,"[32.0, 39.0, 14.0, 7.0, 25.0, 38.0]","[29, 12, 7, 5, 2, 1]"
598,39,"[1494, 9326, 7130]",1,2,6,100,"[24.0, 35.0, 4.0, 30.0, 10.0, 39.0, 5.0, 16.0]","[20, 17, 15, 13, 10, 6, 3, 2]"


In [22]:
df.consistent_r3.mean()

25.888333333333332